## Exploring trade-off between house price and age in Boston dataset
Generally, there is a trade-off between house price and age. But you may want to buy a ralatively new one as cheap as possible.

Then, what kind of building should we look for?

In this tutorial,

1. Train tow machine learning models for house prices and age usign sklearn.

2. Find out optimal input which achieve low cost and ralatively new building using optga.